<a href="https://colab.research.google.com/github/joelmaf/redes_neurais/blob/main/ALUNO_etapa_transformacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problema:  Prever se um cliente vai cancelar sua assinatura

# Estrutura das variáveis


*   **Estado:** O estado de onde o cliente é originário
*   **Tempo de conta:** Número de dias que o cliente está usando os serviços
*   **Código de área:** A área de onde o cliente é originário
*   **Número de telefone:** O número de telefone do cliente
*   **Plano internacional:** O status do plano internacional do cliente
*   **Plano de correio de voz:** O status do plano de correio de voz do cliente
*   **Número de mensagens de correio de voz:** Número de mensagens de correio de voz enviadas pelo cliente
*   **Total de minutos diurnos:** Total de minutos de chamadas feitos por um cliente durante o dia
*   **Total de chamadas diurnas:** Número total de chamadas feitas por um cliente durante o dia
*   **Total de cobranças diurnas:** Valor total cobrado a um cliente **durante** o dia
*   **Total de minutos vespertinos:** Total de minutos de chamadas feitos por um cliente durante a tarde
*   **Total de chamadas vespertinas:** Número total de chamadas feitas por um cliente durante a tarde
*   **Total de cobranças vespertinas:** Valor total cobrado a um cliente durante a tarde
*   **Total de minutos noturnos:** Total de minutos de chamadas feitos por um cliente durante a noite
*   **Total de chamadas noturnas:** Número total de chamadas feitas por um cliente durante a noite
*   **Total de cobranças noturnas:** Valor total cobrado a um cliente durante a noite
*   **Total de minutos internacionais:** Total de minutos de chamadas internacionais feitas por um cliente
*   **Total de chamadas internacionais:** Número total de chamadas internacionais feitas por um cliente
*   **Total de cobranças internacionais:** Valor total cobrado por chamadas internacionais feitas por um cliente
*   **Chamadas ao serviço de atendimento ao cliente:** Número total de chamadas feitas ao serviço de atendimento ao cliente
*   **Cancelamento:** Cancelado ou não

https://github.com/kuldeep1909/Sony-Research-Machine-Learning-Project

In [ ]:
#!pip install pandas
#!pip install numpy
#!pip install matplotlib
#!pip install seaborn
#!pip install scikit-learn
#!pip install jupyter
!pip install shap
!pip install torchmetrics

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
from torch.utils.data import TensorDataset, DataLoader

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from imblearn.over_sampling import ADASYN

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn import set_config
set_config(display='diagram')

import warnings
warnings.filterwarnings('ignore')


In [ ]:
url = 'Sony_data.csv'
df = pd.read_csv(url)
df.head()

#### Divisão em Treino e Teste

**Razões para Dividir em Treino e Teste**

1. Treinar o Modelo: Durante o treinamento, o modelo de aprendizado de máquina utiliza o conjunto de dados de treino para ajustar seus parâmetros internos (pesos, coeficientes, etc.). O objetivo do treinamento é encontrar padrões nos dados e aprender a fazer previsões. Conjunto de treino: Este conjunto de dados é utilizado para treinar o modelo. Ele contém tanto as variáveis de entrada (features) quanto as saídas (rótulos/valores alvo) que o modelo deve prever.

2. Avaliar a Generalização do Modelo: Depois de treinar o modelo, é necessário verificar como ele se comporta com dados que não foram utilizados durante o treinamento. Isso é feito com o conjunto de teste. O conjunto de teste fornece uma avaliação objetiva da capacidade de generalização do modelo, ou seja, sua habilidade de fazer previsões corretas em dados novos e desconhecidos.
Sem essa etapa, você não teria como saber se o modelo está simplesmente memorizando os dados de treino (overfitting) ou se está realmente aprendendo os padrões subjacentes que podem ser aplicados a novos dados.

3. Evitar Overfitting:
Overfitting ocorre quando o modelo se ajusta tão bem aos dados de treino que perde a capacidade de generalizar para novos dados. O modelo pode aprender "ruídos" ou padrões específicos dos dados de treino que não existem no mundo real. Ao avaliar o modelo com dados de teste, que ele nunca viu antes, podemos identificar se ele está superajustando os dados de treino ou se é capaz de generalizar para novos exemplos.

4. Obter uma Estimativa de Performance Real:
Avaliar o modelo com o conjunto de teste fornece uma estimativa realista da sua performance em dados futuros. Isso é importante porque, no mundo real, o modelo será aplicado a dados que não foram vistos durante o treinamento.
Métricas como acurácia, precisão, revocação, F1-score e erro quadrático médio calculadas com o conjunto de teste nos dão uma ideia de quão bem o modelo irá performar em produção.


**Fluxo de Trabalho**

* Treinamento do Modelo: O conjunto de treino é usado para ajustar o modelo, encontrar padrões nos dados e determinar os melhores parâmetros para realizar previsões.

* Validação (opcional): Em muitos casos, além da divisão entre treino e teste, um terceiro conjunto, chamado de validação, é utilizado para afinar os hiperparâmetros do modelo (parâmetros que não são ajustados automaticamente, como a profundidade de uma árvore de decisão ou a taxa de aprendizado em redes neurais).

* Teste do Modelo: O conjunto de teste, que contém dados completamente novos, é utilizado para testar o modelo e medir sua capacidade de fazer previsões corretas em dados desconhecidos.

In [ ]:
# Divisão em treino e teste






print("Conjunto de treino: ", X_train.shape)
print("Conjunto de teste: ", X_test.shape)

# Processamento dos dados

In [ ]:
X_train.head(5)

In [ ]:
X_train.describe(include='object')

In [ ]:
# Corrigir tipo
def correct_type(df_origem):
  df_origem['Codigo_area'] = df_origem['Codigo_area'].astype('object')
  return df_origem

In [ ]:
X_train.describe(include='object')

In [ ]:
# Deletar colunas inúteis
def delete_column(df_origem):
  df_origem = df_origem.drop('Numero_telefone', axis=1)
  return df_origem

In [ ]:
X_train = delete_column(X_train)
X_test = delete_column(X_test)

#### Binning

Binning é o processo de dividir dados contínuos em intervalos ou "bins" (também conhecidos como "caixas" ou "faixas"). Isso é feito para transformar variáveis contínuas (como idade, salário, número de mensagens, etc.) em variáveis categóricas, facilitando a análise e a interpretação dos dados, especialmente em situações onde estamos interessados em categorias ou classes.

Vantagens?
* Simplificação: Ajuda a reduzir a complexidade dos dados contínuos transformando-os em categorias discretas.
* Melhora a interpretabilidade: Em vez de lidar com números contínuos, você pode trabalhar com categorias mais fáceis de entender, como "baixo", "médio" ou "alto".
* Melhor ajuste para alguns algoritmos: Alguns modelos de aprendizado de máquina funcionam melhor com variáveis categóricas, e o binning pode melhorar a performance nesses casos.
* Manuseio de outliers: Pode ajudar a reduzir o impacto de outliers ao agrupar dados em bins.


In [ ]:
#Distribuição original
plt.figure(figsize=(12, 6))

sns.histplot(X_train['Numero_mensagens_voz'], kde=True, color='blue')
plt.title('Antes da transformação', fontsize=14)
plt.xlabel('Número de mensagens de correio de voz', fontsize=12)
plt.ylabel('Frequencia', fontsize=12)

#### Comando pd.cut()

O método cut é usado para segmentar e classificar os dados da coluna ".
* bins: Define os limites dos bins.
* labels: Define os rótulos que serão atribuídos a cada bin.
* right: Indica que os limites do intervalo são exclusivos no lado direito, ou seja, por exemplo, se for FALSE um bin 0-21 inclui os valores de 0 até 20, mas não inclui 21.

In [ ]:
# Binning em dois 2 bins: 0-20, 20+
def binning(df_origem, bins,labels, columns):
  df_origem[columns] = pd.cut(df_origem[columns], bins=bins, labels=labels, right=False)
  return df_origem

In [ ]:
X_train.head(5)

In [ ]:
# Frequência depois do bining
sns.countplot(x='Numero_mensagens_voz', data=X_train, palette='Set2')
plt.title('Depois Binning (Baixo, Alto)', fontsize=14)
plt.xlabel('Binned Número Mensagen Voz', fontsize=12)
plt.ylabel('Frequência', fontsize=12)

In [ ]:
# Binning em 3 bins: 0-2, 3-4, 4+
bins = [0, 3, 5, np.inf]
labels = ['Baixo', 'Médio', 'Alto']

X_train = binning(X_train, bins,labels,'Chamadas_atendimento')
X_test = binning(X_test, bins,labels,'Chamadas_atendimento')

In [ ]:
X_train.head(5)

#### Log Transformation

Log Transformation (Transformação Logarítmica) é uma técnica usada para transformar variáveis de dados, aplicando a função logarítmica a elas. Essa transformação é muito útil em situações onde os dados são altamente assimétricos (ou seja, possuem skewness), ou quando existem outliers que podem distorcer os resultados de análises ou modelagens.

Objetivo da Log Transformation:
* Reduzir Skewness: Muitas variáveis, especialmente em domínios como finanças e economia, têm distribuições altamente assimétricas (skewed), onde a maior parte dos dados se concentra em uma extremidade e há uma cauda longa. A transformação logarítmica pode "achatar" a distribuição, tornando-a mais simétrica e mais parecida com uma distribuição normal, o que é importante para muitas análises estatísticas.
* Reduzir a influência de outliers: Em dados altamente dispersos, valores extremos (outliers) podem ter um grande impacto. A transformação logarítmica reduz o impacto desses valores extremos, fazendo com que eles se ajustem melhor ao restante dos dados.
* Normalização: Em alguns casos, aplicar a transformação logarítmica pode aproximar os dados de uma distribuição normal, o que é uma suposição comum em várias técnicas de análise de dados e aprendizado de máquina.

In [ ]:
sns.histplot(X_train['Total_chamadas_internacionais'], bins=10, kde=True, color='blue')
plt.title('Distribuição original')
plt.xlabel('Total Chamadas Internacionais')
plt.ylabel('Frequência', fontsize=12)

#### np.log1p()

O np.log1p() é uma função da biblioteca NumPy que calcula o logaritmo natural (base e) de um número mais 1. Em outras palavras, np.log1p(x) é equivalente a log(1 + x). Isso é feito para evitar o problema de calcular log(0), que não é definido (o logaritmo de zero não existe). Ao adicionar 1 aos valores, garantimos que a transformação logarítmica possa ser aplicada mesmo quando os dados contêm zeros

In [ ]:
# Aplicando log transformation para reduzir skewness (adicionando 1 para evitar log(0))
def log_transformation(df_origem):
  df_origem['Total_chamadas_internacionais'] = np.log1p(df_origem['Total_chamadas_internacionais'])
  return df_origem

In [ ]:
X_train = log_transformation(X_train)
X_test = log_transformation(X_test)

In [ ]:
X_train.head(5)

In [ ]:
sns.histplot(X_train['Total_chamadas_internacionais'], bins=10, kde=True, color='green')
plt.title('Distribuição transformada (Log Transformation)')
plt.xlabel('Total chamadas internacionais transformadas')
plt.ylabel('Frequência', fontsize=12)

#### Encoding

Encoding (ou codificação) é o processo de converter dados categóricos (geralmente texto ou rótulos) em uma representação numérica que pode ser entendida e processada por algoritmos de aprendizado de máquina. Muitos algoritmos de aprendizado de máquina e modelos estatísticos não conseguem trabalhar diretamente com variáveis categóricas, como textos ou rótulos, pois eles precisam de entradas numéricas. Exemplo de dados categóricos: Colunas com valores como 'sim', 'não', 'alto', 'médio', 'baixo', ou categorias como 'vermelho', 'azul', 'verde'.

A codificação é importante porque a maioria dos modelos de aprendizado de máquina, como regressões lineares, árvores de decisão e redes neurais, só conseguem processar entradas numéricas. Portanto, antes de aplicar qualquer modelo a um conjunto de dados que contenha variáveis categóricas, é necessário converter esses dados para uma forma que o modelo consiga interpretar. Se não fizermos a codificação, o modelo simplesmente não será capaz de processar os dados ou fornecerá resultados incorretos.

Principais Métodos de Encoding:
* Label Encoding (Codificação de Rótulos): Transforma cada categoria em um número inteiro único. Útil quando existe uma relação ordinal entre as categorias (como "baixo", "médio" e "alto"). No entanto, não deve ser usado para dados categóricos sem ordem, pois o modelo pode interpretar as relações numéricas incorretamente. Exemplo: Suponha que temos uma coluna com as categorias ["Alto", "Médio", "Baixo"]. Com o Label Encoding, essas categorias seriam convertidas em [0, 1, 2], ou seja, "Baixo" -> 0, "Médio" -> 1, "Alto" -> 2.

* One-Hot Encoding: Converte categorias em colunas binárias (0 ou 1), onde cada coluna representa uma categoria. Ideal para dados categóricos nominais (sem ordem). A codificação One-Hot evita a criação de uma relação ordinal artificial entre categorias. Exemplo: Se a coluna original contém ["Vermelho", "Verde", "Azul"], o One-Hot Encoding transformaria essa coluna em três colunas: Vermelho, Verde e Azul, onde cada linha teria 1 na coluna correspondente à sua categoria e 0 nas demais.

* Target Encoding (Codificação por Alvo): Substitui as categorias por valores calculados com base na média da variável-alvo (ou outra estatística) para cada categoria. Útil quando a cardinalidade (número de categorias) é alta e existe uma correlação entre as categorias e a variável-alvo. Exemplo: Se estamos tentando prever se um cliente vai cancelar uma assinatura (0 ou 1) e temos uma coluna "País", podemos calcular a média de cancelamento para cada país e substituir o nome do país pela média.

* Binary Encoding: Combina Label Encoding e One-Hot Encoding. Converte as categorias em números inteiros (Label Encoding) e, em seguida, codifica esses números inteiros em binário. Funciona bem quando há muitas categorias, evitando a criação de muitas colunas extras como no One-Hot Encoding.





In [ ]:
X_train.head(5)

In [ ]:
X_train.select_dtypes(include=['object','category']).columns

In [ ]:
X_train_copy = one_hot_encoder(X_train)
X_test_copy = one_hot_encoder(X_test)
X_train_copy.head(5)

In [ ]:
# Aplicando mapping: encoding da variável booleana Cancelamento
def mapping_target(df_origem):
    if isinstance(df_origem, pd.DataFrame):
        boolean_cols = df_origem.select_dtypes(include=['bool']).columns
        df_origem[boolean_cols] = df_origem[boolean_cols].applymap(lambda x: 1 if x else 0)
        return df_origem
    elif isinstance(df_origem, pd.Series) and df_origem.dtype == 'bool':
        return df_origem.map({True: 1, False: 0})

In [ ]:
X_train_copy = mapping_target(X_train_copy)
X_test_copy = mapping_target(X_test_copy)

y_train = mapping_target(y_train)
y_test = mapping_target(y_test)

X_train_copy.head(5)

In [ ]:
y_train

In [ ]:
X_train['Plano_internacional'].unique()

In [ ]:
X_train['Plano_correio'].unique()

In [ ]:
X_train['Numero_mensagens_voz'].unique()

In [ ]:
X_train['Chamadas_atendimento'].unique()

In [ ]:
# Aplicando Label Encoder
def label_encoder(df_origem):
  le = LabelEncoder()
  df_origem['Plano_internacional'] = le.fit_transform(df_origem['Plano_internacional'])
  df_origem['Plano_correio'] = le.fit_transform(df_origem['Plano_correio'])
  df_origem['Chamadas_atendimento'] = le.fit_transform(df_origem['Chamadas_atendimento'])
  df_origem['Numero_mensagens_voz'] = le.fit_transform(df_origem['Numero_mensagens_voz'])
  return df_origem

In [ ]:
X_train = label_encoder(X_train)
X_test = label_encoder(X_test)

In [ ]:
X_train.head()

In [ ]:
X_train['Codigo_area'] = X_train['Codigo_area'].astype(int)
X_test['Codigo_area'] = X_test['Codigo_area'].astype(int)
X_train['Codigo_area'].unique()

In [ ]:
# Aplicando Target Encoding
def target_encoding(X, y, column1, column2):
    X_temp = X.copy()
    X_temp['Cancelamento'] = y

    mean_cancelamento = X_temp.groupby(column1)['Cancelamento'].mean()
    X[column2] = X[column1].map(mean_cancelamento)
    X.drop(columns=[column1], inplace=True)
    return X

In [ ]:
X_train = target_encoding(X_train, y_train,'Estado','Media_cancelamento_por_estado')
X_train = target_encoding(X_train, y_train,'Codigo_area','Media_cancelamento_por_codigo_area')

X_test = target_encoding(X_test, y_test, 'Estado','Media_cancelamento_por_estado')
X_test = target_encoding(X_test, y_test, 'Codigo_area','Media_cancelamento_por_codigo_area')

X_train.head()

#### Feature engineering

A feature engineering (engenharia de características) envolve, por exemplo, a criação de novas variáveis ou atributos derivados dos dados originais. O objetivo é fornecer informações adicionais que possam ajudar o modelo a capturar padrões e fazer previsões mais precisas.

Criar novas colunas pode aumentar a expressividade dos dados:
* Novas colunas podem trazer informações derivadas que não estão explícitas nas colunas originais.
* Melhorar o desempenho do modelo: A criação de novas colunas pode ajudar o modelo a encontrar padrões ou relacionamentos entre variáveis que não seriam evidentes apenas com os dados brutos. Essas variáveis derivadas podem capturar melhor a dinâmica do problema, resultando em um modelo mais preciso.
* Em muitos casos, as variáveis originais podem não ser suficientes para prever adequadamente a variável-alvo. Criar novas features pode complementar os dados e enriquecer a informação disponível para o modelo.
* Capturar relações não lineares: Em vez de trabalhar diretamente com valores absolutos, as relações entre variáveis podem ser mais úteis, mais informativas e permitir que o modelo capture melhor os padrões de consumo do cliente.
* Reduzir a dimensionalidade: Em alguns casos, criar novas colunas pode permitir que você reduza a complexidade dos dados. Ao criar colunas que combinam informações, você pode eliminar a necessidade de várias colunas originais, reduzindo assim a dimensionalidade e tornando o treinamento do modelo mais eficiente.
* Normalizar informações: Algumas colunas, podem variar significativamente, mas a relação entre variáveis pode ser mais consistente e relevante para o modelo. Isso ajuda a capturar o comportamento de forma normalizada.
* Identificar características importantes: Através da criação de novas colunas, pode-se gerar variáveis que sejam mais diretamente relacionadas à variável-alvo. Por exemplo, se estamos tentando prever se um cliente vai cancelar sua assinatura, a eficiência do uso do plano (como custo por minuto) pode ser um preditor mais eficaz do que as cobranças totais ou os minutos usados isoladamente.


In [ ]:
# Criando novas colunas
def feature_engineering(df_origem):
  df_origem['Total_chamadas_diurnas_por_minuto'] = df_origem['Total_cobrancas_diurnas'] / df_origem['Total_minutos_diurnos']
  df_origem['Total_chamadas_vespertinas_por_minuto'] = df_origem['Total_cobrancas_vespertinas'] / df_origem['Total_minutos_vespertinos']
  df_origem['Total_chamadas_noturnas_por_minuto'] = df_origem['Total_cobrancas_noturnas'] / df_origem['Total_minutos_noturnos']
  df_origem['Total_chamadas_internacionais_por_minuto'] = df_origem['Total_cobrancas_internacionais'] / df_origem['Total_minutos_internacionais']
  return df_origem

In [ ]:
X_train = feature_engineering(X_train)
X_test = feature_engineering(X_test)

X_train.head()

#### Feature selection

Feature Selection (ou Seleção de Variáveis ou Seleção de Atributos) é o processo de identificar e selecionar um subconjunto das variáveis ou features mais relevantes em um conjunto de dados que são mais importantes para o desempenho de um modelo de aprendizado de máquina. O objetivo da seleção de variáveis é melhorar a performance do modelo, reduzindo a complexidade, eliminando ruídos, e aumentando a interpretabilidade.


* Redução de Overfitting: Ao reduzir o número de variáveis irrelevantes ou redundantes, você diminui o risco de overfitting (quando o modelo se ajusta demais aos dados de treino e perde a capacidade de generalizar para dados novos).
* Aumento da Performance do Modelo: Eliminar variáveis desnecessárias pode melhorar a performance do modelo, tanto em termos de precisão quanto de velocidade. Modelos mais simples tendem a ser mais rápidos para treinar e predizer, além de serem menos propensos a erro.
* Redução de Tempo de Treinamento: Menos variáveis significa menos dados para processar, o que reduz o tempo de treino do modelo e a quantidade de recursos computacionais necessários.
* Melhoria da Interpretabilidade: Modelos com menos variáveis são mais fáceis de entender e interpretar.
* Remoção de Ruído: Variáveis irrelevantes ou redundantes podem adicionar ruído aos dados, dificultando a identificação de padrões e impactando negativamente o desempenho do modelo.


In [ ]:
X_train.head(5)

In [ ]:
pearson_correlation_matrix = pd.concat([X_train, y_train], axis=1).corr(method='pearson')
mask = np.triu(np.ones_like(pearson_correlation_matrix, dtype=bool))
plt.figure(figsize=(15, 10))
sns.heatmap(pearson_correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.5, annot_kws={"fontsize": 10}, mask=mask)
plt.title('Matriz de Correlação de Pearson')

In [ ]:
# Recuperar as features mais importantes
rf_model = RandomForestClassifier(n_estimators=500)
rf_model.fit(X_train, y_train)

importance = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X_train.columns,'Importancia': importance}).sort_values(by='Importancia', ascending=False)

# Plotting feature importance
plt.figure(figsize=(10, 6))
sns.barplot(x='Importancia', y='Feature', data=feature_importance_df, palette='viridis')
plt.title('Importãncia da Feature na Predição de Cancelamento')
plt.xlabel('Medida de Importância')
plt.ylabel('Features')
plt.show()

In [ ]:
# Calcula a variância de cada coluna (ou variável) em X_train.
# A variância mede a dispersão dos dados em torno da média.
# Quanto maior a variância, mais dispersos são os valores em uma variável.

variance = X_train.var()
variance.sort_values(ascending=False)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

bestfeatures = SelectKBest(k=10)

fit = bestfeatures.fit(X_train,y_train)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X_train.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)
print(featureScores.nlargest(10,'Score'))

In [ ]:
# Retirando algumas colunas
columns_to_drop = ['Total_cobrancas_diurnas', 'Total_cobrancas_noturnas', 'Total_cobrancas_vespertinas',
                   'Total_cobrancas_internacionais', 'Numero_mensagens_voz', 'Total_chamadas_diurnas_por_minuto',
                   'Total_chamadas_vespertinas_por_minuto', 'Total_chamadas_internacionais_por_minuto']

X_train.drop(columns=columns_to_drop, inplace=True)
X_test.drop(columns=columns_to_drop, inplace=True)
X_train.head()

#### Balanceando a classe

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='Cancelamento', data=pd.concat([X_train, y_train], axis=1), palette='Set2')
plt.title('Distribuição da Classe antes de ADASYN')
plt.xlabel('Cancelamento')
plt.ylabel('Contagem')
plt.xticks([0, 1], labels=['Não Cancelado', 'Cancelado'])
plt.show()

In [ ]:
# Aplicando o balanceamento de classes
adasyn = ADASYN(sampling_strategy='minority', random_state=42)
X_resampled, y_resampled = adasyn.fit_resample(X_train, y_train)

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='Cancelamento', data=pd.concat([X_resampled, y_resampled], axis=1), palette='Set2')
plt.title('Distribuição da Classe após ADASYN')
plt.xlabel('Cancelamento')
plt.ylabel('Contagem')
plt.xticks([0, 1], labels=['Não Cancelado', 'Cancelado'])
plt.show()

In [ ]:
before_count = pd.Series(y_train).value_counts()
after_count = pd.Series(y_resampled).value_counts()

comparison_df = pd.DataFrame({
    'Antes': before_count,
   'Depois': after_count
})

comparison_df

In [ ]:
#X_train = X_resampled
#y_train = y_resampled

#### Normalização

Normalização é uma técnica de pré-processamento de dados que ajusta a escala dos valores de uma variável para que todos os valores estejam dentro de um intervalo específico, geralmente entre 0 e 1. O objetivo da normalização é garantir que diferentes variáveis tenham a mesma escala, o que pode ser definitivo para o desempenho de algoritmos de aprendizado de máquina.

A normalização é especialmente útil quando as variáveis do conjunto de dados têm diferentes unidades ou escalas (por exemplo, uma variável representando salários em milhares e outra representando idades). Sem normalização, os algoritmos de aprendizado de máquina podem atribuir mais importância a variáveis com maiores valores numéricos, mesmo que essas variáveis não sejam necessariamente mais importantes.

* Normalização: Escala os dados para que fiquem em um intervalo específico, geralmente entre 0 e 1. Normalização é útil quando você quer manter os valores dentro de um intervalo limitado, como entre 0 e 1.

Fórmula comum:
$$
X_{\text{norm}} = \frac{X - X_{\min}}{X_{\max} - X_{\min}}
$$


* Padronização (Z-score): Transforma os dados para que tenham média 0 e desvio padrão 1. Ela remove a média e escala de acordo com a variabilidade dos dados. Padronização é útil quando você quer manter a distribuição original dos dados, mas precisa alinhar a média e o desvio padrão.

Fórmula comum:

$$
X_{\text{padronizado}} = \frac{X - \mu}{\sigma}
$$



As funções RobustScaler, StandardScaler, FunctionTransformer e MinMaxScaler da biblioteca Scikit-learn são usadas para transformar os dados de diferentes maneiras, dependendo do tipo de pré-processamento que você precisa realizar.

1. RobustScaler: Transforma os dados removendo a mediana e escalando os valores com base no intervalo interquartil (IQR). Ou seja, é robusto em relação a outliers, pois usa a mediana e os percentis 25 e 75 para o escalonamento.

$$
X_{\text{robusto}} = \frac{X - Q2}{Q3 - Q1}
$$


2. StandardScaler: Transforma os dados para que tenham média 0 e desvio padrão 1. Remove a média e divide pelos desvios padrão de cada feature (variável).

3. MinMaxScaler: Escala os dados para um intervalo específico, normalmente entre 0 e 1, mas você pode definir outros intervalos, como [-1, 1].

4. FunctionTransformer: Permite aplicar funções arbitrárias aos dados para transformá-los. Ele permite que você defina uma transformação personalizada sem precisar criar um objeto separado para isso. Isso pode ser útil quando você deseja aplicar transformações específicas, como transformar logaritmos ou outras operações.

In [ ]:
X_train = scaling(X_train)
X_test = scaling(X_test)

In [ ]:
X_train.to_csv('X_train.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_test.to_csv('y_test.csv', index=False)

In [ ]:
X_train.info()

In [ ]:
X_train.head(5)